# Outbound Auth

Outbound Auth allows agents and the AgentCore Gateway to securely access AWS resources and third-party services on behalf of users who have been authenticated and authorized during Inbound Auth. To integrate authorization with an AWS resource or third-party service, it's necessary to configure both Inbound Auth and Outbound Auth.

With just-enough access and secure permission delegation supported by AgentCore Identity, agents can seamlessly and securely access AWS resources and third-party tools such as GitHub, Google, Salesforce, and Slack. Agents can perform actions on these services either on behalf of users or independently, provided there is pre-authorized user consent. Additionally, you can reduce consent fatigue using a secure token vault and create streamlined AI agent experiences.

## Outbound Authentication Configuration

First, you register your client application with third-party providers and then create an Outbound Auth. You specify how you want to validate access to the AWS resource or third-party service or AgentCore Gateway targets. You can use OAuth 2LO/3LO or API keys. With OAuth, you can select from providers that AgentCore Identity provides. In which case you enter the configuration details for the providers from AgentCore Identity. Alternatively, you can supply details for a custom provider. 

When a user wants access to an AWS resource or third-party service or AgentCore Gateway target, the Outbound Auth confirms that the access tokens provided by Incoming Auth are valid and if so, allows access to the resource.

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

## Resource credential providers

This is a component that agent code uses to retrieve credentials of downstream resource servers (e.g., Google, GitHub) to access them, e.g., fetch emails from Gmail, add a meeting to Google Calendar.  It removes the heavy-lifting of agent developers implementing 2LO and 3LO OAuth2 orchestration flows across end-users, agent code, and external authorization servers. AgentCore provides both a custom OAuth2 credential provider and a list of built-in providers such Google, GitHub, Slack, Salesforce with authorization server endpoint and provider-specific parameters pre-filled.
  

Bedrock AgentCore Identity provides OAuth2 and API Key Credential Providers for agent developers to authenticate with external resources that support OAuth2 or API key. In the following example, we will walk you through configuring an API Key credential provider.  An agent can then use the API Key credential provider to retrieve the API key for any agent operations. Please refer to the documentation for the other credential providers.

 ### Creating a resource credential provider.

 Here is an example of creating an OAuth2 resource credential provider. In this example we are creating a credential provider for "Github". This is one of the named resource credential providers. You can also configure this credential provider using the AgentCore console.

```
from bedrock_agentcore.services.identity import IdentityClient

identity_client = IdentityClient("us-east-1")

# Configure Github OAuth2 provider - On-Behalf-Of User
github_provider = identity_client.create_oauth2_credential_provider({
    "name":"<name-of-your-cred-provider>", 
    "credentialProviderVendor":"GithubOauth2",
    "oauth2ProviderConfigInput":{
        "githubOauth2ProviderConfig": {
            "clientId": "<your-github-client-id>",
            "clientSecret": "<your-github-client-secret>"
        }
    }
})
print(github_provider)
```

 ### Retrieving OAuth2 access tokens with the Resource credential provider.

Here is an example of retrieving the OAuth2 access tokens from the credential provider. The agent can then use the access token to interact with resources protected by OAuth. e.g: in our example , this is Github repositories and the scope is read:user. In order to retrieve credentials like access_token from the credential provider, you can decorate your function as shown below.

```
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token

# This annotation helps agent developer to obtain access tokens from external applications
@requires_access_token(
    provider_name="<name-of-your-cred-provider>",
    scopes=["repo", "read:user"],, # Github OAuth2 scopes
    auth_flow="USER_FEDERATION", # On-behalf-of user (3LO) flow
    on_auth_url=on_auth_url,
    force_authentication=True,
)
async def need_token_3LO_async(*, access_token: str):
    print(access_token)

```

Here are the various parameters you can use with the @require_access_token decorator.


| Parameter Name      | Description                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | The credential provider name                                             |
| into                | Parameter name to inject the token into                                  |
| scopes              | OAuth2 scopes to request                                                 |
| on_auth_url	      | Callback for handling authorization URLs                                 |
| auth_flow           | Authentication flow type ("M2M" or "USER_FEDERATION")                    |
| callback_url        | OAuth2 callback URL                                                      |
| force_authentication| Force re-authentication                                                  |
| token_poller        | Custom token poller implementation                                       |

		


# Hosting Strands Agents in Amazon Bedrock AgentCore Runtime 

## Overview


In this tutorial we will develop a agent using Strands agents that can list the private repositories from the users gitHub repo's. We will configure a credential provider to help with credential management with Github. We can use the named provider for Github and modify our agent code to call the credential provider and use the access_token to get the list of private repositories from Github

### Tutorial Architecture

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### Tutorial Details

| Information         | Details                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                           |
| Agent type          | Single                                                                   |
| Agentic Framework   | Strands Agents                                                           |
| LLM model           | Anthropic Claude Sonnet 3.7                                              |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Claude Model |
| Tutorial vertical   | Cross-vertical                                                           |
| Example complexity  | Medium                                                                   |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                             |
| Credential Provider | Type : OAuth2 - Github Provider                                          |


### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Claude models
* Using Strands Agents
* Using AgentCore egress Auth with OAuth2 Github credential provider.


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running
* Enable model access for Claude 3.7 Sonnet v1, i.e. "us.anthropic.claude-3-7-sonnet-20250219-v1:0". For more details refer to [Add or remove access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html)

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

##  Configure Inbound Auth with Cognito as the IDP

Lets provision a Cognito Userpool with an App client and one test user. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server. To do so, we will use the `setup_cognito_user_pool` support function from our `utils` script.

Note: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool, reauthenticate_user

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")

## Configure Github for OAuth2. ( On behalf of user flow )

## Steps to Create an OAuth Client in GitHub (Retrieve client_id and client_secret)

Follow these steps to create an OAuth app in GitHub and retrieve the necessary credentials for use in other applications:

1. **Sign In to GitHub**
   - Go to [github.com](https://github.com) and log in to your account.

2. **Access Developer Settings**
   - Click your profile picture in the upper-right corner.
   - Select **Settings** from the dropdown.
   - In the left sidebar, scroll down and click **Developer settings**.

3. **Go to OAuth Apps**
   - In the left sidebar under Developer settings, click **OAuth Apps**.

4. **Register a New OAuth Application**
   - Click **New OAuth App** (or **Register a new application** if you have not created one before).
   - **Fill in required details:**
     - **Application Name**: Name your app.
     - **Homepage URL**: URL for your app’s homepage. You can use the redirect URI from the [AWS Docs](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity-getting-started-prerequisites.html)<br>
        The redirect URI is : https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback
     - **Application Description** (optional): Add a description for clarity.
     - **Authorization callback URL**: URL where users will be sent after authorization (used for OAuth flow in your application).
        You can find the redirect URI from the [AWS Docs](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity-getting-started-prerequisites.html)<br>
        The redirect URI is : https://bedrock-agentcore.us-east-1.amazonaws.com/identities/oauth2/callback

     - Here is a screen grab showing the Github configuration.
     <div style="text-align:left">
      <img src="images/github_details.png" width="80%"/>
      </div>
   - After entering the details, click **Register application**.

5. **Retrieve Credentials**
   - Once the application is registered, you will be shown the **Client ID** right on the application summary page.
   - To generate the **Client Secret**, click on the available button or link. The secret will be displayed—copy and save it securely; you’ll use it in your application.
     - Note: The client secret is only revealed once or only a few times for security.

***

### Use of Credentials

- You will use the **client_id** and **client_secret** when configuring the Github credential provider.
- Never expose the client secret publicly or share it with users—treat it like a password.

***


#### Configure the Github OAuth2 credential provider.

Modify the following code and replace the following :
1. <your-github-client-id> with the "client id" recorded from Step 5 above
2. <your-github-client-secret> with the "client secret" recorded from Step 5 above.

Important : Please ensure you replace the entire place holder string including the "<" and ">"


Once the client id and client secret are updated, run the below code to create a credentials provider for Github. <br>
Resource credential providers in AgentCore Identity act as intelligent intermediaries that manage the complex relationships between agents, identity providers, and resource servers. Each provider encapsulates the specific endpoint configuration required for a particular service or identity system. The service provides built-in providers for popular services including Google, GitHub, Slack, and Salesforce, with authorization server endpoints and provider-specific parameters pre-configured to reduce development effort. AgentCore Identity supports custom configurations through configurable OAuth2 credential providers that can be tailored to work with any OAuth2-compatible resource server.


In [ ]:
import boto3
import json

# Create credential provider
agentcore_client = boto3.client('bedrock-agentcore-control', region_name=region)

response = agentcore_client.create_oauth2_credential_provider(
    name='github-provider',
    credentialProviderVendor='GithubOauth2',
    oauth2ProviderConfigInput={
        'githubOauth2ProviderConfig': {
            'clientId': "<your-github-client-id>",
            'clientSecret': "<your-github-client-secret>"
        }
    }
)

print(f"✅ Provider created: {response['credentialProviderArn']}")


## Preparing your agent for deployment on AgentCore Runtime

### Strands Agent with a model hosted on Amazon Bedrock
Here is a Strands agent code that includes the following :
1. Creates a new tool called "inspect_github_repos()", to inspect the github repos and list the users private repositories.
2. Uses the Credential provider created in the previous step to fetch the access_token from Github. This includes the user consent flow where the consent user is sent to the user for approval as part of the 3LO flow.
3. The Strands agent calls the tool for any user requests related to github.

In [ ]:
%%writefile github_agent.py

import asyncio
import json
import os
from typing import Optional

import httpx
from bedrock_agentcore import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token
from strands import Agent, tool

# Environment configuration
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Global token storage (could be improved with a proper state management solution)
github_access_token: Optional[str] = None

app = BedrockAgentCoreApp()


@tool
def inspect_github_repos() -> str:
    """Inspect and list the user's private GitHub repositories.

    Returns:
        str: A JSON string containing the list of repositories and their details,
             or an authentication required message.
    """
    global github_access_token

    if not github_access_token:
        return json.dumps({
            "auth_required": True,
            "message": "GitHub authentication is required. Please wait while we set up the authorization.",
            "events": []
        })

    print(f"Using GitHub access token: {github_access_token[:10]}...")

    headers = {"Authorization": f"Bearer {github_access_token}"}

    try:
        with httpx.Client() as client:
            # Get user information
            user_response = client.get("https://api.github.com/user", headers=headers)
            user_response.raise_for_status()
            username = user_response.json().get("login", "Unknown")
            print(f"✅ User: {username}")

            # Search for user's repositories
            repos_response = client.get(
                f"https://api.github.com/search/repositories?q=user:{username}",
                headers=headers
            )
            repos_response.raise_for_status()
            repos_data = repos_response.json()
            print(f"✅ Found {len(repos_data.get('items', []))} repositories")

            repos = repos_data.get('items', [])
            if not repos:
                return f"No repositories found for {username}."

            # Format repository information
            response_lines = [f"GitHub repositories for {username}:\n"]
            
            for repo in repos:
                repo_line = f"📁 {repo['name']}"
                if repo.get('language'):
                    repo_line += f" ({repo['language']})"
                repo_line += f" - ⭐ {repo['stargazers_count']}"
                response_lines.append(repo_line)
                
                if repo.get('description'):
                    response_lines.append(f"   {repo['description']}")
                response_lines.append("")  # Empty line for spacing

            return "\n".join(response_lines)

    except httpx.HTTPStatusError as e:
        return f"GitHub API error: {e.response.status_code} - {e.response.text}"
    except Exception as e:
        return f"Error fetching GitHub repositories: {str(e)}"


class StreamingQueue:
    """Simple async queue for streaming responses."""
    
    def __init__(self):
        self._queue = asyncio.Queue()
        self._finished = False

    async def put(self, item: str) -> None:
        """Add an item to the queue."""
        await self._queue.put(item)

    async def finish(self) -> None:
        """Mark the queue as finished and add sentinel value."""
        self._finished = True
        await self._queue.put(None)

    async def stream(self):
        """Stream items from the queue until finished."""
        while True:
            item = await self._queue.get()
            if item is None and self._finished:
                break
            yield item


# Initialize streaming queue
queue = StreamingQueue()


async def on_auth_url(url: str) -> None:
    """Callback for authentication URL."""
    print(f"Authorization URL: {url}")
    await queue.put(f"Authorization URL: {url}")


def extract_response_text(response) -> str:
    """Extract text content from agent response."""
    if isinstance(response.message, dict):
        content = response.message.get('content', [])
        if isinstance(content, list):
            return "".join(
                item.get('text', '') for item in content 
                if isinstance(item, dict) and 'text' in item
            )
    return str(response.message)


def needs_authentication(response_text: str) -> bool:
    """Check if response indicates authentication is required."""
    auth_keywords = [
        "authentication", "authorize", "authorization", "auth",
        "sign in", "login", "access", "permission", "credential",
        "need authentication", "requires authentication"
    ]
    return any(keyword.lower() in response_text.lower() for keyword in auth_keywords)


async def agent_task(user_message: str) -> None:
    """Execute agent task with authentication handling."""
    global github_access_token
    
    try:
        await queue.put("Begin agent execution")
        
        # Initial agent call
        response = agent(user_message)
        response_text = extract_response_text(response)
        
        # Check if authentication is needed
        if needs_authentication(response_text):
            await queue.put("Authentication required for GitHub access. Starting authorization flow...")
            
            try:
                github_access_token = await need_token_3LO_async(access_token='')
                await queue.put("Authentication successful! Retrying GitHub request...")
                
                # Retry with authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"Authentication error: {auth_error}")
                await queue.put(f"Authentication failed: {str(auth_error)}")
                return
        
        await queue.put(response.message)
        await queue.put("End agent execution")
        
    except Exception as e:
        await queue.put(f"Error: {str(e)}")
    finally:
        await queue.finish()


@requires_access_token(
    provider_name="github-provider",
    scopes=["repo", "read:user"],
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
)
async def need_token_3LO_async(*, access_token: str) -> str:
    """Handle 3LO authentication flow."""
    global github_access_token
    github_access_token = access_token
    return access_token


# Create agent instance
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[inspect_github_repos],
    system_prompt="""You are a GitHub assistant. Use the inspect_github_repos tool to fetch private repositories data.
    The inspect_github_repos tool handles token exchange and proper authentication with the GitHub API 
    to obtain private information for the user."""
)


@app.entrypoint
async def agent_invocation(payload):
    """Main entrypoint for agent invocation."""
    user_message = payload.get(
        "prompt", 
        "No prompt found in input, please guide customer to create a JSON payload with prompt key"
    )
    
    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))
    
    async def stream_with_task():
        """Stream results while ensuring task completion."""
        async for item in queue.stream():
            yield item
        await task  # Ensure task completes
    
    return stream_with_task()


if __name__ == "__main__":
    app.run()


## Deploying the agent to AgentCore Runtime
The CreateAgentRuntime operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

Note: Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

Note : The authorizer_configuration is configured for Inbound Auth with Cognito.

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

discovery_url = cognito_config.get("discovery_url")
client_id = cognito_config.get("client_id")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="github_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_github",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response

## Review the AgentCore configuration

In [ ]:
!cat .bedrock_agentcore.yaml

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

### Add extra required policies to auto-created role
Since we are adding some outbound identity to our agent, we will need to get some API Keys and Secrets that are not available in the auto-created role. To do so, we will need to add some extra permissions to our auto-created IAM role. Let's first get this role and then add those permissions to it.

In [ ]:
import json
import boto3
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey",
                "bedrock-agentcore:GetResourceOauth2Token"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "arn:aws:secretsmanager:*:*:secret:bedrock-agentcore*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

You will notice that the agent calls the "inspect_github_repos" tool and triggers the 3 Legged Outh flow. You will be presented with the Authorization Url. Click on the Authorization url Or copy/paste it in a new browser session/tab to complete the user consent flow.
Once the Authorization completes, The credential provider "github-provider" will fetch the access_token from Github and complete the tool execution to fetch the user's private repo listing.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
bearer_token = reauthenticate_user(cognito_config.get("client_id"))
print(bearer_token)
invoke_response = agentcore_runtime.invoke(
    {"prompt": "What are my private repositories?"}, 
    bearer_token=bearer_token
)
invoke_response

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_role_name
)